# Random Forest for HAMBI data 

## Load libraries

In [1]:
from sklearn.ensemble import RandomForestClassifier
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from datetime import datetime
import joblib

# Set seed
seed = 98

# Set font
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'STIXGeneral'

import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import ListedColormap
from sklearn.metrics import confusion_matrix
from copy import copy

# List all font names
#available_fonts = sorted(set(f.name for f in fm.fontManager.ttflist))
#for font in available_fonts:
#    print(font)

## Import data

In [ ]:
output_folder = '/scratch/project_2006608/Methylation/notebooks/RF_HAMBI'

In [ ]:
data = pd.read_csv("/scratch/project_2006608/Methylation/HAMBI_data/merged_data.tsv", sep="\t", index_col=0, header=0,low_memory=False)

In [ ]:
data.head()

In [ ]:
data['Domain to species'].value_counts(dropna=False)

In [ ]:
len(data['Domain to species'].value_counts(dropna=False))

In [ ]:
len(data.value_counts(dropna=False))

## Random forest classifier

In [ ]:
def clean_data(data: pd.DataFrame) -> pd.DataFrame:
    classes_before = set(data['s'].unique())
    print(f"before filtering number of classes {len(data['s'].value_counts())}")   
    data_filtered = data.dropna(subset=['All'], axis=0)
    data_filtered = data_filtered[(data_filtered['All'] != '') & (data_filtered['All'] != 'Bacteria')]
    data_filtered = data_filtered[(data_filtered.iloc[:, :492] != 0).any(axis=1)]
    print(f"after filtering number of classes {len(data_filtered['s'].value_counts())}")
    classes_after = set(data_filtered['s'].unique())
    dropped_classes = classes_before - classes_after
    if dropped_classes:
        print(f"Dropped classes: {dropped_classes}")
    else:
        print("No classes were dropped.")
    return data_filtered

def filter_out_n(data: pd.DataFrame, column_name: str, n: int) -> pd.DataFrame:
    """Filter out the contigs that have less than n samples in the given column.
    
    Parameters:
    - data (pd.DataFrame): Data frame with the data.
    - column_name (str): Column name to filter out.
    - n (int): Minimum number of samples.
    """
    print(f"before filtering number of classes {len(data[column_name].value_counts())}")
    counts = data[column_name].value_counts()
    classes = counts[counts > n].index
    filtered_data = data[data[column_name].isin(classes)]
    filtered_data = filtered_data.dropna(subset=[column_name], axis=0)
    print(f"after filtering number of classes {len(filtered_data[column_name].value_counts())}")

    return filtered_data

def plot_confusion_matrix(y_true: pd.Series, y_pred: pd.Series, model: RandomForestClassifier, title: str, output_dir: str, target: str):
    """Plot the confusion matrix for the given true and predicted values.
    
    Parameters:
    - y_true (pd.Series): True labels.
    - y_pred (pd.Series): Predicted labels.
    - model (RandomForestClassifier): Trained model.
    - title (str): Title of the plot.
    - output_dir (str): Path to the output directory where the plot will be saved.
    - target (str): Target column.
    """
    cm = confusion_matrix(y_true, y_pred)

    # Replace zeros with np.nan to mask them
    cm_nan = cm.astype(float)
    cm_nan[cm_nan == 0] = np.nan

    # Copy the colormap and set the color for NaNs (bad values)
    base_cmap = copy(plt.get_cmap("YlGnBu"))
    base_cmap.set_bad(color="#FFFFF7")  # lighter color for zero

    # Prepare annotation: show numbers except for NaNs (which become empty strings)
    annot = np.where(np.isnan(cm_nan), "", cm_nan.astype(int).astype(str))
    
    # For Species
    clean_labels = [label.replace("_", " ") for label in model.classes_]
    
    plt.figure(figsize=(50, 50))

    sns.heatmap(cm_nan, cmap=base_cmap, annot=annot, fmt='',
    # For species
        xticklabels=clean_labels, yticklabels=clean_labels,
        #xticklabels=model.classes_, yticklabels=model.classes_,
                cbar=True, annot_kws={"size": 60}, cbar_kws={'label': '', 'shrink': 0.8, 'aspect': 30, 'orientation': 'vertical'})
    
    plt.xticks(rotation = 40, ha='right', fontsize=60, fontstyle='italic')
    plt.yticks(rotation=0, fontsize=60, fontstyle='italic')

    # Edit legend labels
    ax = plt.gca()
    cbar = ax.collections[0].colorbar
    cbar.ax.tick_params(labelsize=60)

    plt.title(f'{title} - Accuracy: {accuracy_score(y_true, y_pred):.2f}', fontsize=110)
    # For species
    xticklabels=clean_labels,
    yticklabels=clean_labels,
    #xticklabels=model.classes_,
    #yticklabels=model.classes_,
    plt.xlabel('Predicted labels', fontsize=80, fontweight='bold')
    plt.ylabel('True labels', fontsize=80, fontweight='bold')
    plt.subplots_adjust(top=0.85)

    plt.tight_layout()
    title = title.replace(' ', '_')
    plt.savefig(os.path.join(output_dir, f'{title}.png'))
    plt.savefig(os.path.join(output_dir, f'{title}.svg'))

    # Edit when you have chosen the number of features
    plt.savefig(os.path.join(output_folder, f'{target}_confusion_mat.png'), bbox_inches='tight')
    plt.savefig(os.path.join(output_folder, f'{target}_confusion_mat.svg'), bbox_inches='tight')
    plt.show()
    
    plt.close()

def plot_train_test_distribution(y_train: pd.Series, y_test: pd.Series, output_dir: str, target: str):
    """Plot the distribution of the train and test sets.
    
    Parameters:
    - y_train (pd.Series): True labels of the train set.
    - y_test (pd.Series): True labels of the test set.
    - output_dir (str): Path to the output directory.
    """
    print("Plotting the distribution")
    train_species_counts = y_train.value_counts()
    test_species_counts = y_test.value_counts()
    
    print("Number of samples in each species in the train set:")
    print(train_species_counts)
    print("\nNumber of samples in each species in the test set:")
    print(test_species_counts)
    num_classes = len(set(y_train.unique()).union(set(y_test.unique())))
    
    width = max(12, num_classes * 0.6)
    height = max(8, num_classes * 0.8)
    
    plt.figure(figsize=(width, height))
    
    categories = pd.concat([y_train, y_test]).value_counts().index
    train_counts = y_train.value_counts().reindex(categories, fill_value=0)
    test_counts = y_test.value_counts().reindex(categories, fill_value=0)
    
    indices = range(len(categories))
    bar_width = 0.4 

    # For species
    labels = ["Citrobacter_B_koseri", "Pseudomonas_E_putida", "Bordetella_avium", "Microvirga_lotononidis",
              "Paracoccus_denitrificans", "Comamonas_testosteroni_C", "Pseudomonas_E_chlororaphis", "Morganella_morganii",
              "Cupriavidus_oxalaticus", "Sphingobium_yanoikuyae", "Aeromonas_caviae"]   
    clean_labels = [label.replace("_", " ") for label in labels]
    
    plt.bar([i - bar_width / 2 for i in indices], train_counts, width=bar_width, color='blue', alpha=0.5, label='Train')
    plt.bar([i + bar_width / 2 for i in indices], test_counts, width=bar_width, color='red', alpha=0.5, label='Test')

    # For species
    plt.xticks(indices, clean_labels, fontsize=16, fontstyle='italic', rotation = 40, ha='right')
    
    # Other than species:
    #plt.xticks(indices, categories, fontsize=16, fontstyle='italic', rotation = 40, ha='right')
    
    plt.yticks(fontsize=16)
    plt.legend(fontsize=18)
    plt.tight_layout()
    plt.title(f'Distribution of train and test set data', fontsize=30)
    plt.subplots_adjust(bottom=0.3)
    
    target = target.replace(' ', '_')
    plt.savefig(os.path.join(output_dir, f'{target}_train_test_distribution.png'), bbox_inches='tight')
    plt.savefig(os.path.join(output_dir, f'{target}_train_test_distribution.svg'))
    plt.show()

def plot_feature_importance(model: RandomForestClassifier, X: pd.DataFrame, target:str, top_n: int = 30, output_dir: str = None):
    """Plot the feature importance of the model.
    
    Parameters:
    - model (RandomForestClassifier): Trained model.
    - X (pd.DataFrame): Data frame with the features.
    - target (str): Target column.
    - top_n (int): Number of top features to plot.
    - output_dir (str): Path to the output directory.
    """

    methylation_colors = {'m6A': '#BF5137',
                          'm4C': '#365E46',
                          'modified_base': '#ABC7C9',
                          'other': 'gray'}

    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1]

    top_indices = indices[:top_n]
    top_features = X.columns[top_indices]

    def get_color(feature_name):
        if 'm6A' in feature_name:
            return methylation_colors['m6A']
        elif 'm4C' in feature_name:
            return methylation_colors['m4C']
        elif 'modified_base' in feature_name:
            return methylation_colors['modified_base']
        else:
            return methylation_colors['other']

    bar_colors = [get_color(feature) for feature in top_features]

    plt.figure(figsize=(20, 10))
    plt.bar(range(top_n), importances[top_indices], align='center', color=bar_colors)
    plt.xticks(range(top_n), X.columns[top_indices], rotation=45, ha='right', fontsize=16)
    plt.yticks(fontsize=16)
    plt.title(f"Top {top_n} features with species", fontsize=50)
    plt.xlim(-0.5, len(top_indices) - 0.5)
    plt.tight_layout()
    output_name = f'{target.replace(" ", "_")}_feature_importance_colors.png'
    plt.savefig(os.path.join(output_dir, f'{target}_feature_importance_colors.png'))
    plt.savefig(os.path.join(output_dir, f'{target}_feature_importance_colors.svg'))
    plt.close()

def train_test_model(df: pd.DataFrame, output_folder: str, target: str):
    """Train and test the random forest classifier.

    Parameters:
    - df (pd.DataFrame): Data frame with the data.
    - output_folder (str): Path to the output directory to save the results for the model plots
    - target (str): Target column.
    """
    print(f"Training the model with target: {target}")
    filtered_data = clean_data(df)
    filtered_data = filter_out_n(filtered_data, target, 10) ###### tätä muokkaa takasin 10
    
    X_DS = filtered_data.iloc[:, :492] 
    y_DS = filtered_data[target]

    X_train_DS, X_test_DS, y_train_DS, y_test_DS = train_test_split(X_DS, y_DS, test_size=0.2, random_state=seed, stratify=y_DS)
    plot_train_test_distribution(y_train_DS, y_test_DS, output_folder, target)
    model = RandomForestClassifier(random_state=seed, n_jobs=-1)
    model.fit(X_train_DS, y_train_DS)

    y_train_pred_DS = model.predict(X_train_DS)
    y_test_pred_DS = model.predict(X_test_DS)

    # For species
    labels = ["Citrobacter_B_koseri", "Pseudomonas_E_putida", "Bordetella_avium", "Microvirga_lotononidis",
              "Paracoccus_denitrificans", "Comamonas_testosteroni_C", "Pseudomonas_E_chlororaphis", "Morganella_morganii",
              "Cupriavidus_oxalaticus", "Sphingobium_yanoikuyae", "Aeromonas_caviae"]
    clean_labels = [label.replace("_", " ") for label in labels]

    plot_confusion_matrix(y_train_DS, y_train_pred_DS, model, f'Train Set with {target}', output_folder, target)
    plot_confusion_matrix(y_test_DS, y_test_pred_DS, model, f'Test Set with {target}', output_folder, target)

    plot_feature_importance(model, X_DS, target, 30, output_folder)

    metrics = pd.DataFrame({
            'Accuracy Train': [accuracy_score(y_train_DS, y_train_pred_DS)],
            'Accuracy Test': [accuracy_score(y_test_DS, y_test_pred_DS)],
            'Precision': [precision_score(y_test_DS, y_test_pred_DS, average='weighted', zero_division=0)],
            'Recall': [recall_score(y_test_DS, y_test_pred_DS, average='weighted', zero_division=0)],
            'F1 Score': [f1_score(y_test_DS, y_test_pred_DS, average='weighted', zero_division=0)]
        })
    
    metrics.to_csv(os.path.join(output_folder, f'{target.replace(" ", "_")}_metrics.tsv'), sep='\t')
    print(f"Metrics for target {target}")
    print(metrics)

    if accuracy_score(y_test_DS, y_test_pred_DS) > 0.7:
        print(f"Model trained with accuracy: {accuracy_score(y_test_DS, y_test_pred_DS)}")
        model_file_path = os.path.join(output_folder, f'{target.replace(" ", "_")}_model.joblib')
        print(f"Saving the model to {model_file_path}")
        joblib.dump(model, model_file_path)
    else:
        print(f"Model trained with accuracy: {accuracy_score(y_test_DS, y_test_pred_DS)}")
    return y_test_DS, y_test_pred_DS
    

In [ ]:
output_folder = '/scratch/project_2006608/Methylation/notebooks/RF_HAMBI'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [ ]:
y_test_DS, y_test_pred_DS = train_test_model(data, output_folder, 's')

In [ ]:
train_test_model(data, output_folder, 'g')

In [ ]:
train_test_model(data, output_folder, 'f')

In [ ]:
train_test_model(data, output_folder, 'o')

In [ ]:
train_test_model(data, output_folder, 'c')

In [ ]:
train_test_model(data, output_folder, 'p')

## Exploring different number of lines in .gff files
### 20, 50, 100, 200, 500

In [ ]:
data = pd.read_csv("/scratch/project_2006608/Methylation/HAMBI_data/merged_data.tsv", sep="\t", index_col=0, header=0,low_memory=False)

In [ ]:
data.head()

In [ ]:
## Delete all rows with < 50 / < 100 / < 200 / < 500 mod counts
## Bring the contig lengths from Puhti:
file_path = '/scratch/project_2006608/Methylation/HAMBI_data/metagenomic_assembly/contigs_mod_counts.tsv'

df_mod_counts = pd.read_csv(file_path, sep='\t', index_col=0, header=0, low_memory=False)
print(df_mod_counts.shape[0])
df_mod_counts.head()

## Append to merged_data.tsv
data_ext = data.copy()
data_mod_counts = pd.concat([data_ext, df_mod_counts], axis=1)
data_mod_counts.head()

#data_mod_counts = data_mod_counts.loc[data_mod_counts['mod_count'] >= 50]
#data_mod_counts = data_mod_counts.loc[data_mod_counts['mod_count'] >= 100]
#data_mod_counts = data_mod_counts.loc[data_mod_counts['mod_count'] >= 200]
data_mod_counts = data_mod_counts.loc[data_mod_counts['mod_count'] >= 500]

print(data_mod_counts.shape[0])

#len(data_mod_counts['Domain to species'].value_counts(dropna=True))
len(data_mod_counts['Domain to species'].value_counts(dropna=False))

## Random forest classifier 
#### Filtered by mod counts
##### > 50
##### > 100
##### > 200
##### > 500

In [ ]:
def clean_data(data_mod_counts: pd.DataFrame) -> pd.DataFrame:
    classes_before = set(data_mod_counts['s'].unique())
    print(f"before filtering number of classes {len(data_mod_counts['s'].value_counts())}")   
    data_mod_counts_filtered = data_mod_counts.dropna(subset=['All'], axis=0)
    data_mod_counts_filtered = data_mod_counts_filtered[(data_mod_counts_filtered['All'] != '') & (data_mod_counts_filtered['All'] != 'Bacteria')]
    data_mod_counts_filtered = data_mod_counts_filtered[(data_mod_counts_filtered.iloc[:, :492] != 0).any(axis=1)]
    print(f"after filtering number of classes {len(data_mod_counts_filtered['s'].value_counts())}")
    classes_after = set(data_mod_counts_filtered['s'].unique())
    dropped_classes = classes_before - classes_after
    if dropped_classes:
        print(f"Dropped classes: {dropped_classes}")
    else:
        print("No classes were dropped.")
    return data_mod_counts_filtered

def filter_out_n(data: pd.DataFrame, column_name: str, n: int) -> pd.DataFrame:
    """Filter out the contigs that have less than n samples in the given column.
    
    Parameters:
    - data (pd.DataFrame): Data frame with the data.
    - column_name (str): Column name to filter out.
    - n (int): Minimum number of samples.
    """
    print(f"before filtering number of classes {len(data_mod_counts[column_name].value_counts())}")
    counts = data_mod_counts[column_name].value_counts()
    classes = counts[counts > n].index
    filtered_data_mod_counts = data_mod_counts[data_mod_counts[column_name].isin(classes)]
    filtered_data_mod_counts = filtered_data_mod_counts.dropna(subset=[column_name], axis=0)
    print(f"after filtering number of classes {len(filtered_data_mod_counts[column_name].value_counts())}")

    return filtered_data_mod_counts

def plot_confusion_matrix(y_true: pd.Series, y_pred: pd.Series, model: RandomForestClassifier, title: str, output_dir: str, target: str):
    """Plot the confusion matrix for the given true and predicted values.
    
    Parameters:
    - y_true (pd.Series): True labels.
    - y_pred (pd.Series): Predicted labels.
    - model (RandomForestClassifier): Trained model.
    - title (str): Title of the plot.
    - output_dir (str): Path to the output directory where the plot will be saved.
    - target (str): Target column.
    """
    cm = confusion_matrix(y_true, y_pred)

    # Replace zeros with np.nan to mask them
    cm_nan = cm.astype(float)
    cm_nan[cm_nan == 0] = np.nan

    # Copy the colormap and set the color for NaNs (bad values)
    base_cmap = copy(plt.get_cmap("YlGnBu"))
    base_cmap.set_bad(color="#FFFFF7")  # lighter color for zero

    # Prepare annotation: show numbers except for NaNs (which become empty strings)
    annot = np.where(np.isnan(cm_nan), "", cm_nan.astype(int).astype(str))
    
    # For Species
    #clean_labels = [label.replace("_", " ") for label in model.classes_]
    
    plt.figure(figsize=(50, 50))

    sns.heatmap(cm_nan, cmap=base_cmap, annot=annot, fmt='',
    # For species
        #xticklabels=clean_labels, yticklabels=clean_labels,
        xticklabels=model.classes_, yticklabels=model.classes_,
                cbar=True, annot_kws={"size": 60}, cbar_kws={'label': '', 'shrink': 0.8, 'aspect': 30, 'orientation': 'vertical'})

    plt.xticks(rotation = 40, ha='right', fontsize=60, fontstyle='italic')
    plt.yticks(rotation=0, fontsize=60, fontstyle='italic')

    # Edit legend labels
    ax = plt.gca()
    cbar = ax.collections[0].colorbar
    cbar.ax.tick_params(labelsize=60)

    plt.title(f'{title} - Accuracy: {accuracy_score(y_true, y_pred):.2f}', fontsize=110)
    # For species
    #xticklabels=clean_labels,
    #yticklabels=clean_labels,
    xticklabels=model.classes_,
    yticklabels=model.classes_,
    plt.xlabel('Predicted labels', fontsize=80, fontweight='bold')
    plt.ylabel('True labels', fontsize=80, fontweight='bold')
    plt.subplots_adjust(top=0.85)

    plt.tight_layout()
    title = title.replace(' ', '_')
    plt.savefig(os.path.join(output_dir, f'{title}.png'))
    plt.savefig(os.path.join(output_dir, f'{title}.svg'))

    # Edit when you have chosen the number of features
    plt.savefig(os.path.join(output_folder, f'{target}_confusion_mat.png'), bbox_inches='tight')
    plt.savefig(os.path.join(output_folder, f'{target}_confusion_mat.svg'), bbox_inches='tight')
    plt.show()
    
    plt.close()

def plot_train_test_distribution(y_train: pd.Series, y_test: pd.Series, output_dir: str, target: str):
    """Plot the distribution of the train and test sets.
    
    Parameters:
    - y_train (pd.Series): True labels of the train set.
    - y_test (pd.Series): True labels of the test set.
    - output_dir (str): Path to the output directory.
    """
    print("Plotting the distribution")
    train_species_counts = y_train.value_counts()
    test_species_counts = y_test.value_counts()
    
    print("Number of samples in each species in the train set:")
    print(train_species_counts)
    print("\nNumber of samples in each species in the test set:")
    print(test_species_counts)
    num_classes = len(set(y_train.unique()).union(set(y_test.unique())))
    
    width = max(12, num_classes * 0.6)
    height = max(8, num_classes * 0.8)
    
    plt.figure(figsize=(width, height))
    
    categories = pd.concat([y_train, y_test]).value_counts().index
    train_counts = y_train.value_counts().reindex(categories, fill_value=0)
    test_counts = y_test.value_counts().reindex(categories, fill_value=0)
    
    indices = range(len(categories))
    bar_width = 0.4 

    # For species
    #labels = ["Citrobacter_B_koseri", "Pseudomonas_E_putida", "Bordetella_avium", "Microvirga_lotononidis",
    #          "Paracoccus_denitrificans", "Comamonas_testosteroni_C", "Pseudomonas_E_chlororaphis", "Morganella_morganii",
    #          "Cupriavidus_oxalaticus", "Sphingobium_yanoikuyae", "Aeromonas_caviae"]   
    #clean_labels = [label.replace("_", " ") for label in labels]
    
    plt.bar([i - bar_width / 2 for i in indices], train_counts, width=bar_width, color='blue', alpha=0.5, label='Train')
    plt.bar([i + bar_width / 2 for i in indices], test_counts, width=bar_width, color='red', alpha=0.5, label='Test')

    # For species
    #plt.xticks(indices, clean_labels, fontsize=16, fontstyle='italic', rotation = 40, ha='right')
    
    # Other than species:
    plt.xticks(indices, categories, fontsize=16, fontstyle='italic', rotation = 40, ha='right')
    
    plt.yticks(fontsize=16)
    plt.legend(fontsize=18)
    plt.tight_layout()
    plt.title(f'Distribution of train and test set data', fontsize=30)
    plt.subplots_adjust(bottom=0.3)
    
    target = target.replace(' ', '_')
    plt.savefig(os.path.join(output_dir, f'{target}_train_test_distribution.png'), bbox_inches='tight')
    plt.savefig(os.path.join(output_dir, f'{target}_train_test_distribution.svg'))
    plt.show()

def plot_feature_importance(model: RandomForestClassifier, X: pd.DataFrame, target:str, top_n: int = 30, output_dir: str = None):
    """Plot the feature importance of the model.
    
    Parameters:
    - model (RandomForestClassifier): Trained model.
    - X (pd.DataFrame): Data frame with the features.
    - target (str): Target column.
    - top_n (int): Number of top features to plot.
    - output_dir (str): Path to the output directory.
    """
    
    methylation_colors = {'m6A': '#BF5137',
                          'm4C': '#365E46',
                          'modified_base': '#ABC7C9',
                          'other': 'gray'}

    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1]

    top_indices = indices[:top_n]
    top_features = X.columns[top_indices]

    def get_color(feature_name):
        if 'm6A' in feature_name:
            return methylation_colors['m6A']
        elif 'm4C' in feature_name:
            return methylation_colors['m4C']
        elif 'modified_base' in feature_name:
            return methylation_colors['modified_base']
        else:
            return methylation_colors['other']

    bar_colors = [get_color(feature) for feature in top_features]

    plt.figure(figsize=(20, 10))
    plt.bar(range(top_n), importances[top_indices], align='center', color=bar_colors)
    plt.xticks(range(top_n), X.columns[top_indices], rotation=45, ha='right', fontsize=16)
    plt.yticks(fontsize=16)
    plt.title(f"Top {top_n} features with species", fontsize=50)
    plt.xlim(-0.5, len(top_indices) - 0.5)
    plt.tight_layout()
    output_name = f'{target.replace(" ", "_")}_feature_importance_colors.png'
    plt.savefig(os.path.join(output_dir, f'{target}_feature_importance_colors.png'))
    plt.savefig(os.path.join(output_dir, f'{target}_feature_importance_colors.svg'))
    plt.close()

def train_test_model(df: pd.DataFrame, output_folder: str, target: str):
    """Train and test the random forest classifier.

    Parameters:
    - df (pd.DataFrame): Data frame with the data.
    - output_folder (str): Path to the output directory to save the results for the model plots
    - target (str): Target column.
    """
    print(f"Training the model with target: {target}")
    filtered_data_mod_counts = clean_data(df)
    filtered_data_mod_counts = filter_out_n(filtered_data_mod_counts, target, 10)
    
    X_DS = filtered_data_mod_counts.iloc[:, :492] 
    y_DS = filtered_data_mod_counts[target]

    X_train_DS, X_test_DS, y_train_DS, y_test_DS = train_test_split(X_DS, y_DS, test_size=0.2, random_state=seed, stratify=y_DS)
    plot_train_test_distribution(y_train_DS, y_test_DS, output_folder, target)
    model = RandomForestClassifier(random_state=seed, n_jobs=-1)
    model.fit(X_train_DS, y_train_DS)

    y_train_pred_DS = model.predict(X_train_DS)
    y_test_pred_DS = model.predict(X_test_DS)

    # For species
    #labels = ["Citrobacter_B_koseri", "Pseudomonas_E_putida", "Bordetella_avium", "Microvirga_lotononidis",
    #          "Paracoccus_denitrificans", "Comamonas_testosteroni_C", "Pseudomonas_E_chlororaphis", "Morganella_morganii",
    #          "Cupriavidus_oxalaticus", "Sphingobium_yanoikuyae", "Aeromonas_caviae"]
    #clean_labels = [label.replace("_", " ") for label in labels]

    plot_confusion_matrix(y_train_DS, y_train_pred_DS, model, f'Train Set with {target}', output_folder, target)
    plot_confusion_matrix(y_test_DS, y_test_pred_DS, model, f'Test Set with {target}', output_folder, target)

    plot_feature_importance(model, X_DS, target, 30, output_folder)

    metrics = pd.DataFrame({
            'Accuracy Train': [accuracy_score(y_train_DS, y_train_pred_DS)],
            'Accuracy Test': [accuracy_score(y_test_DS, y_test_pred_DS)],
            'Precision': [precision_score(y_test_DS, y_test_pred_DS, average='weighted', zero_division=0)],
            'Recall': [recall_score(y_test_DS, y_test_pred_DS, average='weighted', zero_division=0)],
            'F1 Score': [f1_score(y_test_DS, y_test_pred_DS, average='weighted', zero_division=0)]
        })
    
    metrics.to_csv(os.path.join(output_folder, f'{target.replace(" ", "_")}_metrics.tsv'), sep='\t')
    print(f"Metrics for target {target}")
    print(metrics)

    if accuracy_score(y_test_DS, y_test_pred_DS) > 0.7:
        print(f"Model trained with accuracy: {accuracy_score(y_test_DS, y_test_pred_DS)}")
        model_file_path = os.path.join(output_folder, f'{target.replace(" ", "_")}_model.joblib')
        print(f"Saving the model to {model_file_path}")
        joblib.dump(model, model_file_path)
    else:
        print(f"Model trained with accuracy: {accuracy_score(y_test_DS, y_test_pred_DS)}")
    return y_test_DS, y_test_pred_DS
    

In [ ]:
#output_folder = '/scratch/project_2006608/Methylation/notebooks/RF_HAMBI_above50'
#output_folder = '/scratch/project_2006608/Methylation/notebooks/RF_HAMBI_above100'
#output_folder = '/scratch/project_2006608/Methylation/notebooks/RF_HAMBI_above200'
output_folder = '/scratch/project_2006608/Methylation/notebooks/RF_HAMBI_above500'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [ ]:
y_test_DS, y_test_pred_DS = train_test_model(data_mod_counts, output_folder, 's')

In [ ]:
len(np.unique(y_test_pred_DS))

In [ ]:
train_test_model(data_mod_counts, output_folder, 'g')

In [ ]:
train_test_model(data_mod_counts, output_folder, 'f')

In [ ]:
train_test_model(data_mod_counts, output_folder, 'o')

In [ ]:
train_test_model(data_mod_counts, output_folder, 'c')

In [ ]:
train_test_model(data_mod_counts, output_folder, 'p')